# Notebook Tutoriel sur une utilisation de Tensorflow pour un réseau neuronal quantique

### Paul SKAF, Fabien VERDIER, Adrien OVIGNE

Ce notebook est notre rendu du projet final, accompagné de notre rapport écrit, visant à expliquer une des utilisations de tensorflow sur un réseau neuronal quantique. 

On propose ici de récupérer une grande série d'image de "3" et de "6" et d'utiliser le machine learning pour apprendre au programme à distinguer quels images sont des "3" et lesquels sont des "6".
L'objectif final est de comparer l'efficacité de ce machine learning en utilisant un réseau quantique et un réseau classique.

Ce code est trouvable en ligne sur le site de Tensorflow : https://www.tensorflow.org/quantum/tutorials/mnist.










# Code

In [0]:
pip install -q tensorflow==2.1.0

In [0]:
pip install -q tensorflow-quantum cirq==0.7.0


On importe premièrement les différents modules nécéssaire à la réalisation du programme.

In [0]:
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import sympy
import numpy as np
import seaborn as sns
import collections

# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit

## 1. Chargement des données

### 1.1 Chargement des images brutes

On utilise Keras pour interagir avec Tensorflow. La bibliothèque Keras permet d'interagir avec les algorithmes de réseaux de neurones profonds et de machine learning. On l'utilise ici pour charger les images depuis la banque de données MNIST.

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# On redimensionne la plage des images de [0,255] à [0.0,1.0] 
x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0

print("Number of original training examples:", len(x_train))
print("Number of original test examples:", len(x_test))

On filtre ici les images qu'on à reçu pour ne garder que les "3" et les "6".
On va également convertir le label "y" en booléen : Vrai si c'est un 3 et Faux si c'est un 6 

In [0]:
def filter_36(x, y):
    keep = (y == 3) | (y == 6)
    x, y = x[keep], y[keep]
    y = y == 3
    return x,y

In [0]:
x_train, y_train = filter_36(x_train, y_train)
x_test, y_test = filter_36(x_test, y_test)

print("Number of filtered training examples:", len(x_train))
print("Number of filtered test examples:", len(x_test))

On montre le premier exemple (la première image donc le premier élément dans x_train, associé au premier label y_train)  : 

In [0]:
print(y_train[0])

plt.imshow(x_train[0, :, :, 0])
plt.colorbar()

### 1.2 Redimensionnement des images

Etant donné que chaque Qubit va être associé à un pixel, une image de taille 28x28 est beaucoup trop grande pour un ordinateur quantique. On redimensionne donc en 4x4 pixels pour avoir 16 qubits sur lesquels travailler.

In [0]:
x_train_small = tf.image.resize(x_train, (4,4)).numpy()
x_test_small = tf.image.resize(x_test, (4,4)).numpy()

On remontre la première image redimensionnée : 

In [0]:
print(y_train[0])

plt.imshow(x_train_small[0,:,:,0], vmin=0, vmax=1)
plt.colorbar()

### 1.3 Suppresion des images contradictoires

Cette fonction remove_contradicting va supprimer les images qui possèdent deux labels en même temps.
On utilise une collection DefaultDict qui appartient à la bibliothèque Collections de python : [Collections Python](https://docs.python.org/2/library/collections.html) pour les gérer.


Source : <a href="https://arxiv.org/pdf/1802.06002.pdf" class="external">Farhi et al.</a> pour la détection des numéros (Section 3.3)

In [0]:
def remove_contradicting(xs, ys):
    mapping = collections.defaultdict(set)
    for x,y in zip(xs,ys):
       mapping[tuple(x.flatten())].add(y)
    
    new_x = []
    new_y = []
    for x,y in zip(xs, ys):
      labels = mapping[tuple(x.flatten())]
      if len(labels) == 1:
          new_x.append(x)
          new_y.append(list(labels)[0])
      else:
          # On jette les images qui possèdent les deux labels (True et False)
          pass
    
    num_3 = sum(1 for value in mapping.values() if True in value)
    num_6 = sum(1 for value in mapping.values() if False in value)
    num_both = sum(1 for value in mapping.values() if len(value) == 2)

    print("Number of unique images:", len(mapping.values()))
    print("Number of 3s: ", num_3)
    print("Number of 6s: ", num_6)
    print("Number of contradictory images: ", num_both)
    print()
    print("Initial number of examples: ", len(xs))
    print("Remaining non-contradictory examples: ", len(new_x))
    
    return np.array(new_x), np.array(new_y)

Cette prévention n'est pas effective à 100% et n'empêchera pas à la suite du programme de recevoir tout de même des images contenant 2 labels.

In [0]:
x_train_nocon, y_train_nocon = remove_contradicting(x_train_small, y_train)

### 1.4 Codage des images sous un circuit quantique

La première étape est donc d'encoder chaque pixel de l'image avec un Qubit.
On fixe un THRESHOLD (seuil) de 0,5. Si la valeur d'un pixel est supérieur au seuil alors on va pouvoir la traiter et la transformer en float.

In [0]:
THRESHOLD = 0.5

x_train_bin = np.array(x_train_nocon > THRESHOLD, dtype=np.float32)
x_test_bin = np.array(x_test_small > THRESHOLD, dtype=np.float32)

Retirer des images contradictoires à ce stade ne nous laissera qu'avec une centaine d'image, car il n'y a plus assez d'information sur les images pour distinguer nettement les 3 ou 6, ce qui n'est pas suffisant pour traiter les données correctement dans la suite du programme.

In [0]:
_ = remove_contradicting(x_train_bin, y_train_nocon)

Les qubits qui sont maintenant associés aux pixels dépassant le seuil, sont transformé au travers d'une Porte $X$ .

In [0]:
def convert_to_circuit(image):
    """Encode truncated classical image into quantum datapoint."""
    values = np.ndarray.flatten(image)
    qubits = cirq.GridQubit.rect(4, 4)
    circuit = cirq.Circuit()
    for i, value in enumerate(values):
        if value:
            circuit.append(cirq.X(qubits[i]))
    return circuit


x_train_circ = [convert_to_circuit(x) for x in x_train_bin]
x_test_circ = [convert_to_circuit(x) for x in x_test_bin]

Voilà le circuit associé à la première image affiché dans le haut du programme 

On remarque que seul 2 pixels dépassent le seuil de 0,5, celui en (2,2) et celui en (3,1), d'où les deux transformations dans le circuit :

In [0]:
SVGCircuit(x_train_circ[0])

On compare maintenant numériquement juste pour vérifier où se situe les pixels dépassant le seuil :

In [0]:
bin_img = x_train_bin[0,:,:,0]
indices = np.array(np.where(bin_img)).T
indices

On finit cette partie en convertissant les objets circuits Cirq en tenseur pour TensorFlow : 

In [0]:
x_train_tfcirc = tfq.convert_to_tensor(x_train_circ)
x_test_tfcirc = tfq.convert_to_tensor(x_test_circ)

## 2. Réseau de neurone quantique

Il y a très peu de données sur les réseaux de neurones quantiques qui classifient les images. La classification est basé sur le qubit de lecture. <a href="https://arxiv.org/pdf/1802.06002.pdf" class="external">Farhi et al.</a> propose un modèle avec un qubit de lecture.

### 2.1 Construction du circuit

Le circuit est construit à l'aide d'une approche par couche. La classe *CircuitLayerBuiler* permet de créer un circuit et d'y ajouter des couches en ajoutant une porte quantique à tous les qubits.

In [0]:
class CircuitLayerBuilder():
    def __init__(self, data_qubits, readout):
        self.data_qubits = data_qubits
        self.readout = readout
    
    def add_layer(self, circuit, gate, prefix):
        for i, qubit in enumerate(self.data_qubits):
            symbol = sympy.Symbol(prefix + '-' + str(i))
            circuit.append(gate(qubit, self.readout)**symbol)

Voici un exemple de circuit à 4 qubits et un qubit de lecture. Nous affichons le circuit résultant avec la fonction SVGCircuit.

In [0]:
demo_builder = CircuitLayerBuilder(data_qubits = cirq.GridQubit.rect(4,1),
                                   readout=cirq.GridQubit(-1,-1))

circuit = cirq.Circuit()
demo_builder.add_layer(circuit, gate = cirq.XX, prefix='xx')
SVGCircuit(circuit)

Nous allons maintenant construire un circuit à 2 couches, mais avec 16 qubits, le nombre de pixels de l'image. Les deux couches une couches avec deux portes X et une autre avec deux portes Z.

In [0]:
def create_quantum_model():
    """Create a QNN model circuit and readout operation to go along with it."""
    data_qubits = cirq.GridQubit.rect(4, 4)  # a 4x4 grid.
    readout = cirq.GridQubit(-1, -1)         # a single qubit at [-1,-1]
    circuit = cirq.Circuit()
    
    # Prepare the readout qubit.
    circuit.append(cirq.X(readout))
    circuit.append(cirq.H(readout))
    
    builder = CircuitLayerBuilder(
        data_qubits = data_qubits,
        readout=readout)

    # Then add layers (experiment by adding more).
    builder.add_layer(circuit, cirq.XX, "xx1")
    builder.add_layer(circuit, cirq.ZZ, "zz1")

    # Finally, prepare the readout qubit.
    circuit.append(cirq.H(readout))

    return circuit, cirq.Z(readout)

In [0]:
model_circuit, model_readout = create_quantum_model()

### 2.2 Adaptation du circuit à tfq-keras

Nous construisons le réseau en empilant les couches avec `tf.keras.Sequential`. L'input n'est pas quantique, mais l'output oui. Avec `tfq.layers.PQC`, on initialise les paramètres et on les gère de manière native avec Keras.

Pour classer les images, <a href="https://arxiv.org/pdf/1802.06002.pdf" class="external">Farhi et al.</a> propose de regarder la valeur de readout qui doit se situer entre -1 et 1. Si cette valeur est inférieur à zéro, alors on a 6, sinon on a 3.

In [0]:
# Build the Keras model.
model = tf.keras.Sequential([
    # The input is the data-circuit, encoded as a tf.string
    tf.keras.layers.Input(shape=(), dtype=tf.string),
    # The PQC layer returns the expected value of the readout gate, range [-1,1].
    tfq.layers.PQC(model_circuit, model_readout),
])

On décrit ensuite la méthode d'apprentissage avec la fonction `compile`.

Nous utilisons `Hinge loss` comme fonction coût. Cette fonction attend des valeurs entre -1 et 1. Donc on fait passer `y_train_hinge` et `y_test_hinge` entre -1 et 1 très simplement.

In [0]:
y_train_hinge = 2.0*y_train_nocon-1.0
y_test_hinge = 2.0*y_test-1.0

Nous devons ensuite faire notre propre fonction `hinge_accuracy` pour gérer les valeurs entre -1 et 1 car `tf.losses.BinaryAccuracy(threshold=0.0)` attend en entrée un booléen et n'est donc pas utilisable avec la fonction coût hinge loss.

Lors de la compilation du modèle, on précise la fonction personnalisée pour la métrique.

In [0]:
def hinge_accuracy(y_true, y_pred):
    y_true = tf.squeeze(y_true) > 0.0
    y_pred = tf.squeeze(y_pred) > 0.0
    result = tf.cast(y_true == y_pred, tf.float32)

    return tf.reduce_mean(result)

In [0]:
model.compile(
    loss=tf.keras.losses.Hinge(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[hinge_accuracy])

In [0]:
print(model.summary())

### Entrainement du circuit quantique

Il faut maintenant entrainer le circuit quantique avec le set de données à notre disposition. Le faire avec toutes les données prend beaucoup de temps (3-4h). Pour que ce temps dure moins longtemps, on peut entrainer le modéle circuit sur moins de données (on met `NUM_EXAMPLES=500` ci-dessous par exemple), mais les performances du circuit seront moins bonnes.

`EPOCHS` donne le nombre d'itérations sur le set de données, `BATCH_SIZE` le nombre de paramètres.

In [0]:
EPOCHS = 3
BATCH_SIZE = 32

NUM_EXAMPLES = len(x_train_tfcirc)

In [0]:
x_train_tfcirc_sub = x_train_tfcirc[:NUM_EXAMPLES]
y_train_hinge_sub = y_train_hinge[:NUM_EXAMPLES]

On arrive à une précision supérieur à 85% en entrainant ce circuit avec toutes les données.
On évalue le circuit avec la fonction `evaluate`. `qnn_results[1]` nous donne la précision du circuit.

In [0]:
qnn_history = model.fit(
      x_train_tfcirc_sub, y_train_hinge_sub,
      batch_size=32,
      epochs=EPOCHS,
      verbose=1,
      validation_data=(x_test_tfcirc, y_test_hinge))

qnn_results = model.evaluate(x_test_tfcirc, y_test)

Note: La précision est la moyenne des précisions à chaque itérations.

## 3. Réseau de Neurone Classique
Un réseau de neurone classique peut facilement dépasser un réseau de neurone quantique sur ce genre d'échantillon.
Dans l'exemple qui suit, le réseau de neurone classique va traiter les images en entière (28x28 pixels) et va facilement converger vers 100% de précision en peu de temps.


In [0]:
def create_classical_model():
    # A simple model based off LeNet from https://keras.io/examples/mnist_cnn/
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(32, [3, 3], activation='relu', input_shape=(28,28,1)))
    model.add(tf.keras.layers.Conv2D(64, [3, 3], activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(1))
    return model


model = create_classical_model()
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

model.summary()

In [0]:
model.fit(x_train,
          y_train,
          batch_size=128,
          epochs=1,
          verbose=1,
          validation_data=(x_test, y_test))

cnn_results = model.evaluate(x_test, y_test)

Le modèle du dessus a presque 1 200 000 paramètres. 
Pour pouvoir comparer équitablement avec le modèle quantique, on va utiliser un modèle à 37 paramètres :


In [0]:
def create_fair_classical_model():
    # A simple model based off LeNet from https://keras.io/examples/mnist_cnn/
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(4,4,1)))
    model.add(tf.keras.layers.Dense(2, activation='relu'))
    model.add(tf.keras.layers.Dense(1))
    return model


model = create_fair_classical_model()
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

model.summary()

In [0]:
model.fit(x_train_bin,
          y_train_nocon,
          batch_size=128,
          epochs=20,
          verbose=2,
          validation_data=(x_test_bin, y_test))

fair_nn_results = model.evaluate(x_test_bin, y_test)

## 4. Comparaison

Une entrée à plus haute résolution et un modèle plus puissant rendent ce problème facile pour un réseau de neurone classique. Alors qu'un modèle classique de puissance similaire (~32 paramètres) s'entraîne avec une précision similaire en une fraction de temps. Le réseau neuronal classique surpasse facilement le réseau neuronal quantique. Pour les données classiques, il est difficile de battre un réseau de neurones classique.

In [0]:
qnn_accuracy = qnn_results[1]
cnn_accuracy = cnn_results[1]
fair_nn_accuracy = fair_nn_results[1]

sns.barplot(["Quantum", "Classical, full", "Classical, fair"],
            [qnn_accuracy, cnn_accuracy, fair_nn_accuracy])